In [1]:
import os
import pandas as pd
from PIL import Image, ImageDraw, ImageFont
import numpy as np
from shapely.geometry import Polygon, LineString
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import json
from PIL import Image, ImageDraw, ImageTk
import tkinter as tk
from tkinter import messagebox
from tkinter import simpledialog
import ast
import json
from sklearn.linear_model import LogisticRegression
import pickle
%matplotlib inline

In [2]:
font_path = r"D:\codes\summer_intern_time_machine\SegDetector\dejavu-sans\ttf\DejaVuSans.ttf"  # Ensure this path is correct
font = ImageFont.truetype(font_path, 16)

In [2]:
def find_file_recursive(directory, filename):
    for root, dirs, files in os.walk(directory):
        if filename in files:
            return os.path.join(root, filename)
    return None

def is_clockwise(poly):
    # Calculate the signed area using the shoelace formula
    x, y = poly.exterior.coords.xy
    area = 0.5 * sum(x[i] * y[i + 1] - x[i + 1] * y[i] for i in range(-1, len(x) - 1))
    return area < 0

def plot_polygons(original: list(), simplified: list()):
    fig, ax = plt.subplots(dpi=300)
    
    # Draw original
    for i, orig in enumerate(original):
        x, y = orig.exterior.xy
        if i == 0:
            ax.plot(x, y, 'b-', label='Original Polygon', linewidth=1.3, alpha=0.7)
        else:
            ax.plot(x, y, 'b-', linewidth=1.3, alpha=0.7)

    # Draw bezier polylines
    for i, beziers in enumerate(simplified):
        
        for j, bezier in enumerate(beziers):
            x, y = bezier.xy
            if (i == 0) and (j == 0):
                ax.plot(x, y, 'r-', label='Bezier curves', linewidth=1., alpha=0.6)
            else:
                ax.plot(x, y, 'r-', linewidth=1., alpha=0.6)

    ax.set_title('Polygon Simplification')
    ax.legend()
    ax.set_aspect('equal')
    plt.xlabel('X coordinate')
    plt.ylabel('Y coordinate')
    plt.grid(True)
    plt.show()
    
def angle_between(v1, v2):
    '''Computes the angle at a certain point on the polygon based on the two adjacent vertices'''
    v1_u = v1 / np.linalg.norm(v1)
    v2_u = v2 / np.linalg.norm(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))

def get_bezier_polylines(poly):
    if not poly.is_valid or poly.is_empty:
        return None
    
    coords = list(poly.exterior.coords)
    coords = [coords[-2]] + coords
    n = len(coords)
    angles = []
    
    for i in range(1, n-1):
        p0 = np.array(coords[i - 1])  # Last point
        p1 = np.array(coords[i])      # This point
        p2 = np.array(coords[(i + 1) % n])  # Next point
        
        v1 = p1 - p0  # Vect from the last point to the present one
        v2 = p2 - p1  # Vect from the present point to the next one
        
        angle = angle_between(v1, v2)
        angles.append(np.degrees(angle))

    main_angles = sorted(np.argsort(angles)[-4:] + 1)
    
    l1 = LineString(coords[main_angles[0]:main_angles[1]+1])
    l2 = LineString(coords[main_angles[1]:main_angles[2]+1])
    l3 = LineString(coords[main_angles[2]:main_angles[3]+1])
    l4 = LineString(coords[main_angles[3]:-1] + coords[1:main_angles[0]+1])

    segments = np.array([l1, l2, l3, l4], dtype=object)
    longest_segments_arg = np.argsort([l1.length, l2.length, l3.length, l4.length])[-2:]
    beziers = segments[longest_segments_arg]

    return beziers

def get_bezier_polylines_single_word(poly):
    if not poly.is_valid or poly.is_empty:
        return None
    
    coords = list(poly.exterior.coords)
    coords = [coords[-2]] + coords
    n = len(coords)
    angles = []
    
    for i in range(1, n-1):
        p0 = np.array(coords[i - 1])  # Last point
        p1 = np.array(coords[i])      # This point
        p2 = np.array(coords[(i + 1) % n])  # Next point
        
        v1 = p1 - p0  # Vect from the last point to the present one
        v2 = p2 - p1  # Vect from the present point to the next one
        
        angle = angle_between(v1, v2)
        angles.append(np.degrees(angle))

    main_angles = sorted(np.argsort(angles)[-4:] + 1)
    
    l1 = LineString(coords[main_angles[0]:main_angles[1]+1])
    l2 = LineString(coords[main_angles[1]:main_angles[2]+1])
    l3 = LineString(coords[main_angles[2]:main_angles[3]+1])
    l4 = LineString(coords[main_angles[3]:-1] + coords[1:main_angles[0]+1])

    segments = np.array([l1, l2, l3, l4], dtype=object)
    
    def compute_centroid(line):
        coordinates = list(line.coords)
        avg_x = sum(point[0] for point in coordinates) / len(coordinates)
        avg_y = sum(point[1] for point in coordinates) / len(coordinates)
        return (avg_x, avg_y)
    
    y1 = compute_centroid(l1)[1]
    y2 = compute_centroid(l2)[1]
    y3 = compute_centroid(l3)[1]
    y4 = compute_centroid(l4)[1]
    
    # print(y1, y2, y3, y4)
    
    top_line_arg = np.argsort([y1, y2, y3, y4])[-1]
    low_line_arg = np.argsort([y1, y2, y3, y4])[0]
    
    beziers = segments[[top_line_arg, low_line_arg]]
    
    # print(beziers)

    return beziers


def simplify_vertices(x):    
    # try:
    #     if len(x["text_cased"]) <= 2 and x["group_id"] == -1:
    #         poly = Polygon(ori_vertices.copy() + [ori_vertices[0]]).buffer(0)
    #         if not is_clockwise(poly):
    #             poly = Polygon(np.flip(list(poly.exterior.coords), 0))
    #         lines = get_bezier_polylines_single_word(poly)
    #         new_vertices = [[(x, y) for x, y in lines[0].coords], [(x, y) for x, y in lines[1].coords]]
    #         return new_vertices
    # except:
    #     pass
    #     # print(x["text_cased"])
    #     # print(x["vertices"])
    try:
        if len(x["text"]) <= 2 and x["group_id"] == -1:
            try:
                ori_vertices = x["vertices_corrected"]
                poly = Polygon(ori_vertices.copy() + [ori_vertices[0]]).buffer(0)
                if not is_clockwise(poly):
                    poly = Polygon(np.flip(list(poly.exterior.coords), 0))
                lines = get_bezier_polylines_single_word(poly)
                new_vertices = [[(x, y) for x, y in lines[0].coords], [(x, y) for x, y in lines[1].coords]]
                return new_vertices
            except:
                ori_vertices = x["vertices"]
                poly = Polygon(ori_vertices.copy() + [ori_vertices[0]]).buffer(0)
                if not is_clockwise(poly):
                    poly = Polygon(np.flip(list(poly.exterior.coords), 0))
                lines = get_bezier_polylines_single_word(poly)
                new_vertices = [[(x, y) for x, y in lines[0].coords], [(x, y) for x, y in lines[1].coords]]
                return new_vertices
        else:
            ori_vertices = x["vertices"]
            poly = Polygon(ori_vertices.copy() + [ori_vertices[0]]).buffer(0).simplify(3)
            if not is_clockwise(poly):
                poly = Polygon(np.flip(list(poly.exterior.coords), 0))
            lines = get_bezier_polylines(poly)
            new_vertices = [[(x, y) for x, y in lines[0].coords], [(x, y) for x, y in lines[1].coords]]
            return new_vertices

    except:
        print(x["text"])
        return None
    

from shapely.geometry import Polygon
from tqdm import tqdm

def intersection_over_union(polygon1_points, polygon2_points):
    """
    Calculate the Intersection over Union (IoU) of two polygons.

    Parameters:
    - polygon1_points: List of (x, y) tuples representing the first polygon.
    - polygon2_points: List of (x, y) tuples representing the second polygon.

    Returns:
    - iou: Intersection over Union metric.
    """
    poly1 = Polygon(polygon1_points)
    poly2 = Polygon(polygon2_points)
    
    if not poly1.is_valid or not poly2.is_valid:
        return 0

    intersection_area = poly1.intersection(poly2).area
    union_area = poly1.union(poly2).area
    
    iou = intersection_area / union_area if union_area != 0 else 0
    
    return iou


def match_polygons(polygon_set1, polygon_set2, iou_threshold=0.5):
    """
    Match polygons from two sets based on IoU.

    Parameters:
    - polygon_set1: List of polygons (each polygon is a list of (x, y) tuples) representing the first set.
    - polygon_set2: List of polygons (each polygon is a list of (x, y) tuples) representing the second set.
    - iou_threshold: Threshold for considering two polygons as a match.

    Returns:
    - matches: List of tuples, each containing the indices of matched polygons (index in set1, index in set2).
    """
    matches = []
    used_indices = set()

    for i, poly1 in tqdm(enumerate(polygon_set1)):
        best_iou = 0  # best intersection over union
        best_match_index = -1
        for j, poly2 in enumerate(polygon_set2):
            if j in used_indices:
                continue
            iou = intersection_over_union(poly1, poly2)
            if iou > best_iou:
                best_iou = iou
                best_match_index = j
        
        if best_iou >= iou_threshold and best_match_index != -1:
            matches.append((i, best_match_index))
            used_indices.add(best_match_index)
    
    return matches


def plot_matches(img_name, annotation_polygons, model_polygons, matches):
    
    file_path = find_file_recursive("./new_annotation/maps_res/", img_name)
    dir_path = os.path.dirname(file_path)
    
    image = Image.open(file_path)
    
    draw = ImageDraw.Draw(image)
    
    matched_annot_id = [match[0] for match in matches]
    matched_model_id = [match[1] for match in matches]
    unmatched_annot_id = [i for i in range(len(annotation_polygons)) if i not in matched_annot_id]
    unmatched_model_id = [i for i in range(len(model_polygons)) if i not in matched_model_id]
    
    for match in matches:
        color = tuple(np.random.choice(range(256), size=3))
        draw.line(annotation_polygons[match[0]], fill=color, width=2)
        draw.line(model_polygons[match[1]], fill=color, width=2)
        draw.text(model_polygons[match[1]][-1], text=df_model.iloc[match[1]].text, font=font)
    
    for unmatched_annot in unmatched_annot_id:
        draw.line(annotation_polygons[unmatched_annot], fill="black", width=2)
    
    for unmatched_model in unmatched_model_id:
        draw.line(model_polygons[unmatched_model], fill="white", width=2)
    
    # image.show()
    image.save(dir_path + "/" + img_name.split('.')[0]+"_matching_res.png")
    

def plot_propagation(img_name, df_merged):
    
    file_path = find_file_recursive("./new_annotation/maps_res/", img_name)
    dir_path = os.path.dirname(file_path)
    
    image = Image.open(file_path)

    draw = ImageDraw.Draw(image)
    
    df_merged_valid = df_merged.dropna(subset=['vertices_corrected'])
    for polygon, text in zip(df_merged_valid['vertices_corrected'].tolist(), df_merged_valid['text'].tolist()):
        draw.line(polygon, fill='green', width=2)
        draw.text(polygon[-1], text, fill="green", font=font)
    
    
    df_merged_na = df_merged[df_merged['vertices_corrected'].isna()]
    for polygon, text in zip(df_merged_na['vertices'].tolist(), df_merged_na['text'].tolist()):
        draw.line(polygon, fill='red', width=2)
        draw.text(polygon[0], text, fill="red", font=font)
    
    # image.show()
    image.save(dir_path + "/" + img_name.split('.')[0]+"_propagated_res.png")
    

def changecase(word):
    ocr_text = str(word["OCR_text"])
    annotated_text = str(word["text"])
    returned_str = annotated_text
            
    for i, char in enumerate(ocr_text):
        if char.isupper():
            returned_str = annotated_text[:i] + annotated_text[i:].capitalize()
            try:
                if ocr_text[i+1].isupper():
                    returned_str = annotated_text.upper()
                    break
            except IndexError:
                break
    
    return returned_str


def get_features(x):
    try:
        line1 = x['vertices_simplified'][0]
        line2 = x['vertices_simplified'][1]
        feature = [line2[0][0] - line1[0][0], # x of first point of upper to the first point of lower
               line2[0][1] - line1[0][1], # y of first point of upper to the first point of lower
               line2[-1][0] - line1[-1][0], # y of first point of upper to the first point of lower
               line2[-1][1] - line1[-1][1], # y of first point of upper to the first point of lower
               ]
        return feature
    except:
        return None
    

def correct_order(x, model):
    try:
        line1 = x["vertices_simplified"][0]
        line2 = x["vertices_simplified"][1]
        y_pred = model.predict([x["feature"]]).item()
        if y_pred == 1:
            return [line1, line2]
        else:
            return [line2, line1]
    except:
        return None
    
    
from sklearn.cluster import AgglomerativeClustering, KMeans

def get_textual_direction(vertices):
    
    diff = vertices[0][-1] - vertices[0][0]
    
    return diff


def normalize(vectors):
    return vectors / np.linalg.norm(vectors, axis=1, keepdims=True)


def cosine_similarity_matrix(vectors):
    normalized_vectors = normalize(vectors)
    return np.dot(normalized_vectors, normalized_vectors.T)


def group_vectors(vectors):
    similarity_matrix = cosine_similarity_matrix(vectors)
    if np.all(similarity_matrix > 0):
        # All vectors have positive cosine similarity, they are in one group
        return np.zeros(len(vectors), dtype=int)
    else:
        # Some vectors have negative cosine similarity, we need to split into two groups
        distance_matrix = 1 - similarity_matrix
        clustering = AgglomerativeClustering(n_clusters=2, affinity='precomputed', linkage='complete')
        # clustering = KMeans(n_clusters=2)
        labels = clustering.fit_predict(distance_matrix)
        return labels


def unify_group_direction(x):
    
    crop_name, group_id = x.name
    if group_id == -1:
        return x
    
    polylines = x["vertices_simplified_order_corrected"].tolist()
    vectors = []
    try:
        for lines in polylines:
            vectors.append((np.array(lines[0][-1]) - np.array(lines[0][0])).tolist())
    except:
        # print(polylines)
        return x
    
    labels = group_vectors(np.array(vectors))
    # print(labels)
    
    if 1 not in labels:
        return x
    else:
        # print(labels)
        num_ones = np.sum(labels)
        num_zeros = len(labels) - num_ones
        if num_ones > num_zeros:
            for i in np.where(labels==0)[0].tolist():
                polylines[i] = [polylines[i][1], polylines[i][0]]
        elif num_ones < num_zeros:
            for i in np.where(labels==1)[0].tolist():
                polylines[i] = [polylines[i][1], polylines[i][0]]
        new_x = x.copy()
        new_x["vertices_simplified_order_corrected"] = polylines
        
        return new_x
    

def calculate_polyline_centroid(lines):
    
    centroid_x = (np.mean(np.array(lines[0])[:, 0]) + np.mean(np.array(lines[1])[:, 0])) / 2
    centroid_y = (np.mean(np.array(lines[0])[:, 1]) + np.mean(np.array(lines[1])[:, 1])) / 2
    
    return np.array([centroid_x, centroid_y])


def identify_group_direction(x):
    
    polylines = x["vertices_simplified_order_corrected"].tolist()
    try:
        first_text_direction = normalize([np.array(polylines[0][0][-1]) - np.array(polylines[0][0][0])])
        
        centroid_second = calculate_polyline_centroid(polylines[1])
        centroid_first = calculate_polyline_centroid(polylines[0])
    
        base_direction = normalize([np.array(centroid_second) - np.array(centroid_first)])
        angle = np.arccos(np.clip(np.dot(base_direction[0], first_text_direction[0]), -1.0, 1.0))
        if angle > (3/4) * np.pi:
            # print(base_direction[0], first_text_direction[0])
            for i, lines in enumerate(polylines):
                polylines[i] = [lines[1], lines[0]]
            new_x = x.copy()
            new_x["vertices_simplified_order_corrected"] = polylines
            
            return new_x
        else:
            return x
    except:
        return x
    

def plot_final(img_name, df):
    
    file_path = find_file_recursive("./new_annotation/maps_res/", img_name)
    dir_path = os.path.dirname(file_path)
    
    image = Image.open(file_path)
    
    draw = ImageDraw.Draw(image)
    
    corrected_vertices = df[df["crop_name"]==img_name]["vertices_simplified_order_corrected"].tolist()
    original_vertices = df[df["crop_name"]==img_name]["vertices"].tolist()
    for i, vertices in enumerate(corrected_vertices):
        if vertices:
            draw.line(vertices[0] + vertices[1], fill='orange', width=2)
            draw.line(vertices[0], fill="blue", width=2)
            draw.line(vertices[1], fill="blue", width=2)
        else:
            vertices = original_vertices[i]
            draw.line(vertices, fill='red', width=2)
        
    # image.show()
    image.save(dir_path + "/" + img_name.split('.')[0]+"_final_corr.png")

In [4]:
df_annotation = pd.read_pickle("./new_annotation/final/cvat_annotations_20240806_pandas.pkl")
df_annotation['vertices'] = df_annotation['vertices'].apply(lambda x: [(point[0], point[1]) for point in x])

In [3]:
for crop in df_annotation["crop_name"].unique().tolist():
    
    try:
        print("Processing {}".format(crop))
    
        df = df_annotation[df_annotation["crop_name"] == crop]
        file_path = find_file_recursive("./new_annotation/maps_res/", crop)
        dir_path = os.path.dirname(file_path)
        df_model = pd.read_json(dir_path + "/flattened_detections.json")
        
        annotation_polygons = df['vertices'].tolist()
        model_polygons = df_model.apply(lambda x: [(x.polygon_x[i], x.polygon_y[i]) for i in range(len(x.polygon_x))], axis=1).tolist()
        
        iou_threshold = 0.6
        matches = match_polygons(annotation_polygons, model_polygons, iou_threshold)
        
        # plot_matches(crop, annotation_polygons, model_polygons, matches)
        
        df_matches = None
        for match in matches:
            df_matches = pd.concat((df_matches, pd.DataFrame({"vertices": [df.iloc[match[0]].vertices],
                                                             "vertices_corrected": [model_polygons[match[1]]],
                                                              "OCR_text": [df_model.iloc[match[1]]["text"]]})))
            
        df['vertices'] = df['vertices'].apply(str)
        df_matches['vertices'] = df_matches['vertices'].apply(str)
        df_merged = pd.merge(df, df_matches, on='vertices', how='outer')
        df_merged['vertices'] = df_merged['vertices'].apply(ast.literal_eval)
        df_merged['vertices'] = df_merged['vertices'].apply(lambda x: [(point[0], point[1]) for point in x])
        
        df_merged['text'] = df_merged.apply(changecase, axis=1)
        
        # plot_propagation(crop, df_merged)
        
        df_merged["vertices_simplified"] = df_merged.apply(simplify_vertices, axis=1)
        
        df_merged["feature"] = df_merged.apply(get_features, axis=1)
        
        with open("ord_cor.pkl", "rb") as f:
            model = pickle.load(f)
            
        df_merged["vertices_simplified_order_corrected"] = df_merged.apply(correct_order, args=(model,), axis=1)
        
        # step1: unify the direction based on majority vote
        df_merged_modified = df_merged.groupby(["crop_name", "group_id"]).apply(unify_group_direction).reset_index(drop=True)
        
        # step2: identify the direction based on base direction
        grouped = df_merged_modified.groupby(['crop_name', 'group_id'])
        modified_groups = []
        for (crop_name, group_id), group in grouped:
            # Sort each group by sequence_id before applying the custom function
            if group_id == -1:
                modified_groups.append(group)
                continue
            group = group.sort_values(by='sequence_order')
            modified_group = identify_group_direction(group)
            modified_groups.append(modified_group)
        df_merged_identified = pd.concat(modified_groups).reset_index(drop=True)
        
        plot_final(crop, df_merged_identified)
        
        df_merged_identified.to_pickle(dir_path + "/" + crop + ".pkl")
    
    except:
        continue

NameError: name 'df_annotation' is not defined

In [73]:
with open("annotations.json", 'r') as f:
    data = json.load(f)
    
# Extract features and labels
features = [sample["features"] for sample in data]
labels = [sample["label"] for sample in data]

# Convert to numpy arrays
X = np.array(features)
y = np.array(labels)

model = LogisticRegression()
model.fit(X, y)

with open("ord_cor.pkl", "wb") as f:
    pickle.dump(model, f)

LogisticRegression()

In [16]:
import shutil

for root, dirs, files in os.walk("./new_annotation/maps_res"):
    for file in files:
        if "final_corr" in file:
            shutil.copy(os.path.join(root, file), os.path.join("new_annotation/vis", file))

In [4]:
def points_to_str(pts):
    
    pts_str = []
    upper, lower = pts[0], pts[1]
    for pt in lower[::-1]:
        pt_str = str(pt[0]) + "," + str(pt[1])
        pts_str.append(pt_str)
    for pt in upper[::-1]:
        pt_str = str(pt[0]) + "," + str(pt[1])
        pts_str.append(pt_str)
    return ";".join(pts_str)

    
def get_image_dimensions(image_path):
    with Image.open(image_path) as img:
        width, height = img.size
    return width, height


def get_annotation_dicts_from_df(df):
    
    annotations = []
    
    for row in df.iterrows():
        if row[1][-1]:
            points = row[1][-1]
            points_str = points_to_str(points)
        else:
            points = row[1][0]
            points_str = ";".join([str(pt[0]) + "," + str(pt[1]) for pt in points])
        group_id = row[1][4]
        sequence_order = row[1][5]
        if group_id == -1:
            group_id = 1
            sequence_order = None
        else:
            group_id = group_id
            sequence_order = sequence_order
        transcription = row[1][1]
        truncated = row[1][3]
        illegible = row[1][2]
        
        word_dict = {"transcription": transcription,
                "points": points_str,
                "group_id": group_id,
                "sequence_order": sequence_order,
                "truncated": truncated,
                "illegible": illegible,
                "z_order": -1}
        
        annotations.append(word_dict)
    
    return annotations


def convert_pkl_to_json(df):
    
    img_name = df["crop_name"].tolist()[0]
    img_path = find_file_recursive("./new_annotation/maps_res", img_name)
    width, height = get_image_dimensions(img_path)
    annotations = get_annotation_dicts_from_df(df)
    
    img_annotation = {"image_name": img_name,
                      "width": width,
                      "height": height,
                      "polygons": annotations}
    
    return img_annotation
    
    # with open(img_path+".json", 'w', encoding="utf-8") as json_file:
    #     json.dump(img_annotation, json_file, indent=4)
        

import json
import xml.etree.ElementTree as ET
import xml.dom.minidom

def json_to_xml(json_obj):
    # Create the root element
    annotations = ET.Element("annotations")
    
    # Add version element
    version = ET.SubElement(annotations, "version")
    version.text = "1.1"
    
    # Add meta element
    meta = ET.SubElement(annotations, "meta")
    job = ET.SubElement(meta, "job")
    ET.SubElement(job, "id").text = "12"
    ET.SubElement(job, "size").text = "1"
    ET.SubElement(job, "mode").text = "annotation"
    ET.SubElement(job, "overlap").text = "0"
    ET.SubElement(job, "bugtracker").text = ""
    ET.SubElement(job, "created").text = "2024-04-07 14:09:14.625571+00:00"
    ET.SubElement(job, "updated").text = "2024-08-13 20:36:33.809220+00:00"
    ET.SubElement(job, "subset").text = "default"
    ET.SubElement(job, "start_frame").text = "0"
    ET.SubElement(job, "stop_frame").text = "0"
    ET.SubElement(job, "frame_filter").text = ""
    
    segments = ET.SubElement(job, "segments")
    segment = ET.SubElement(segments, "segment")
    ET.SubElement(segment, "id").text = "12"
    ET.SubElement(segment, "start").text = "0"
    ET.SubElement(segment, "stop").text = "0"
    ET.SubElement(segment, "url").text = "http://10.95.16.20:8080/api/jobs/12"
    
    owner = ET.SubElement(job, "owner")
    ET.SubElement(owner, "username").text = "kaede.johnson"
    ET.SubElement(owner, "email").text = "kaede.johnson@epfl.ch"
    
    ET.SubElement(job, "assignee").text = ""
    
    labels = ET.SubElement(job, "labels")
    label = ET.SubElement(labels, "label")
    ET.SubElement(label, "name").text = "text"
    ET.SubElement(label, "color").text = "#fa3253"
    ET.SubElement(label, "type").text = "polygon"
    
    attributes = ET.SubElement(label, "attributes")
    attribute_names = ["illegible", "sequence_order", "transcription", "truncated"]
    for attr_name in attribute_names:
        attribute = ET.SubElement(attributes, "attribute")
        ET.SubElement(attribute, "name").text = attr_name
        ET.SubElement(attribute, "mutable").text = "False"
        input_type = "checkbox" if attr_name in ["illegible", "truncated"] else "text"
        ET.SubElement(attribute, "input_type").text = input_type
        ET.SubElement(attribute, "default_value").text = "false" if input_type == "checkbox" else ""
        ET.SubElement(attribute, "values").text = "false" if input_type == "checkbox" else ""
    
    ET.SubElement(meta, "dumped").text = "2024-08-14 10:10:50.238232+00:00"
    
    # Create the image element
    image = ET.SubElement(annotations, "image", id="0", name=json_obj["image_name"],
                          width=str(json_obj["width"]), height=str(json_obj["height"]))
    
    # Create polygon elements based on the JSON polygons
    for polygon in json_obj["polygons"]:
        polygon_element = ET.SubElement(image, "polygon", label="text", source="manual", occluded="0",
                                        points=polygon["points"], z_order=str(polygon["z_order"]),
                                        group_id=str(polygon["group_id"]))
        
        for attr in ["transcription", "illegible", "truncated", "sequence_order"]:
            attribute_element = ET.SubElement(polygon_element, "attribute", name=attr)
            if attr == "sequence_order":
                if str(polygon[attr]) == "None":
                    attribute_element.text = ""
                else:
                    attribute_element.text = str(polygon[attr])
            else:
                attribute_element.text = str(polygon[attr])
    
    # Convert the ElementTree to a string
    xml_str = ET.tostring(annotations, encoding='unicode', method='xml')
    
    return xml_str

# df = pd.read_pickle("./new_annotation/maps_res/catherwood_1835/catherwood_1835.jpeg.pkl")
# json_data = convert_pkl_to_json(df)
# print(json_data)

In [ ]:
for crop in df_annotation["crop_name"].unique().tolist():
    
    try:
        print("Converting {} to xml".format(crop))
    
        file_path = find_file_recursive("./new_annotation/maps_res/", crop)
        dir_path = os.path.dirname(file_path)
        df = pd.read_pickle(dir_path + "/" +  crop+ ".pkl")
    
        json_data = convert_pkl_to_json(df)
        xml_str = json_to_xml(json_data)
        dom = xml.dom.minidom.parseString(xml_str)
        pretty_xml_as_string = dom.toprettyxml()
    
        with open(dir_path + "/" +  crop+ ".xml", "w", encoding="utf-8") as xml_file:
            xml_file.write(pretty_xml_as_string)
    
    except:
        print("Failed converting {}".format(crop))
        continue

In [5]:
import xml.etree.ElementTree as ET

def xml_to_df(crop_name):

    tree = ET.parse('./new_annotation/maps_res/{}/annotations.xml'.format(crop_name.split(".")[0]))
    root = tree.getroot()
    
    # for each crop annotated
    for image in root.findall('image'):
    
        # access all polygons (aka words)
        polygons = image.findall('.//polygon')
        polygon_list = []
        for polygon in polygons:
            tmp_dict = {}
            tmp_dict['vertices'] = [tuple(map(float, pair.split(','))) for pair in polygon.get('points').split(';')]
            #print(polygon.find(".//attribute[@name='sequence_order']").text)
            if polygon.find(".//attribute[@name='sequence_order']").text is not None:
                tmp_dict['group_id'] = int(polygon.get('group_id'))
            else:
                tmp_dict['group_id'] = -1    
            for attribute in polygon.findall('.//attribute'):
                if attribute.get('name') == 'transcription':
                    tmp_dict['text'] = attribute.text
                else:
                    tmp_dict[attribute.get('name')] = attribute.text
            polygon_list.append(tmp_dict)
        polygon_df = pd.DataFrame(polygon_list).sort_values(by=['group_id', 'sequence_order']).reindex(columns=['vertices', 'text', 'illegible', 'truncated', 'group_id', 'sequence_order'])
    
        return polygon_df
    

def plot_and_save_ultimate(img_name, df):
    
    file_path = find_file_recursive("./new_annotation/maps_res/", img_name)
    dir_path = os.path.dirname(file_path)
    
    image = Image.open(file_path)
    
    draw = ImageDraw.Draw(image)
    
    corrected_vertices = df[df["crop_name"]==img_name]["vertices_final"].tolist()
    texts = df[df["crop_name"]==img_name]["text_final"].tolist()
    for vertices, text in zip(corrected_vertices, texts):
        try:
            draw.line(vertices[0] + vertices[1], fill='orange', width=2)
            draw.line(vertices[0], fill="blue", width=2)
            draw.line(vertices[1], fill="blue", width=2)
            draw.text(vertices[1][-1], text=text)
        except:
            continue
        
    # image.show()
    image.save(dir_path + "/" + img_name.split('.')[0]+"_ultimate_corr.png")
    
    df.to_pickle(dir_path + "/" + img_name.split('.')[0]+"_ultimate_corr.png.pkl")

In [12]:
filename = "unknown_1907_crop_2468_1674.jpg"
df_new = xml_to_df(filename)
df_old = pd.read_pickle(find_file_recursive("new_annotation/maps_res", "{}.pkl".format(filename)))

old_polygons = df_old["vertices"].tolist()
new_polygons = df_new["vertices"].tolist()

iou_threshold = 0.1
matches = match_polygons(old_polygons, new_polygons, iou_threshold)

print(len(matches))
print(len(df_old))

print(set(range(len(df_old))) - set([match[0] for match in matches]))
print(set(range(len(df_old))) - set([match[1] for match in matches]))
print(set([match[0] for match in matches]) - set([match[0] for match in matches]).intersection(set([match[1] for match in matches])))
print(set([match[1] for match in matches]) - set([match[0] for match in matches]).intersection(set([match[1] for match in matches])))

148it [00:01, 92.15it/s] 

148
148
set()
set()
set()
set()


In [13]:
df_matches = None
for match in matches:
    df_matches = pd.concat((df_matches, pd.DataFrame({"vertices": [df_old.iloc[match[0]]["vertices"]],
                                                     "vertices_new": [df_new.iloc[match[1]]["vertices"]],
                                                      "text_new": [df_new.iloc[match[1]]["text"]]})))

df_old['vertices'] = df_old['vertices'].apply(str)
df_matches['vertices'] = df_matches['vertices'].apply(str)
df_merged = pd.merge(df_old, df_matches, on='vertices', how='inner')
df_merged['vertices'] = df_merged['vertices'].apply(ast.literal_eval)
df_merged['vertices'] = df_merged['vertices'].apply(lambda x: [(point[0], point[1]) for point in x])

In [14]:
def get_final_vertices(x):
    # if x["vertices_simplified_order_corrected"] is None:
    #     print(x["text_new"])
    #     print(x["vertices_new"])
    if x["text_new"].endswith("/") and not x["text_new"].endswith("//"):
        n_vertices = len(x["vertices_new"])
        # assert n_vertices % 2 == 0
        if n_vertices % 2 != 0:
            print(x["text_new"])
            print(x["vertices"])
        final_vertices = [x["vertices_new"][:n_vertices//2], x["vertices_new"][n_vertices//2:]]
        # assert is_clockwise(Polygon(final_vertices[0] + final_vertices[1]))
        return final_vertices
    elif x["text_new"].endswith("//"):
        final_vertices = [x["vertices_simplified_order_corrected"][0][::-1],
                          x["vertices_simplified_order_corrected"][1][::-1]]
        # assert is_clockwise(Polygon(final_vertices[0] + final_vertices[1]))
        return final_vertices
    else:
        final_vertices = [x["vertices_simplified_order_corrected"][1][::-1],
                          x["vertices_simplified_order_corrected"][0][::-1]]
        # assert is_clockwise(Polygon(final_vertices[0] + final_vertices[1]))
        return final_vertices

df_merged["vertices_final"] = df_merged.apply(get_final_vertices, axis=1)
df_merged["text_final"] = df_merged.apply(lambda x: x["text_new"].strip("/"), axis=1)
display(df_merged)
plot_and_save_ultimate(filename, df_merged)

,vertices,text,illegible,truncated,group_id,sequence_order,crop_name,vertices_corrected,OCR_text,vertices_simplified,feature,vertices_simplified_order_corrected,vertices_new,text_new,vertices_final,text_final
0,"[(147.41, 211.41), (143.81, 211.16), (140.6, 2...",5,false,false,-1,None,unknown_1907_crop_2468_1674.jpg,"[(123.0235900879, 215.4983062744), (123.738998...",5,"[[(127.2040710449, 245.2281951904), (127.96829...","[18.441848754899993, -33.0678405761, -26.06521...","[[(127.2040710449, 245.2281951904), (127.96829...","[(123.02, 215.5), (123.74, 215.36), (124.7, 21...",5,"[[(123.0235900879, 215.4983062744), (123.73899...",5
1,"[(91.3, 589.41), (84.18, 589.38), (77.19, 589....",Tower,false,false,-1,None,unknown_1907_crop_2468_1674.jpg,"[(41.5743255615, 589.8125457764), (43.78880310...",Tower,"[[(91.3, 589.41), (42.86, 589.29)], [(42.55, 6...","[-48.75, 18.379999999999995, 48.59, 18.4100000...","[[(42.55, 607.79), (91.45, 607.7)], [(91.3, 58...","[(42.86, 589.29), (91.3, 589.41), (91.45, 607....",Tower,"[[(42.86, 589.29), (91.3, 589.41)], [(91.45, 6...",Tower
2,"[(151.78, 586.46), (148.2, 586.41), (144.94, 5...",6,false,false,-1,None,unknown_1907_crop_2468_1674.jpg,"[(127.4473495483, 588.6155090332), (127.814918...",6,"[[(130.5075378418, 616.1682281494), (130.69633...","[18.366958618199988, -30.91835021969996, -24.4...","[[(130.5075378418, 616.1682281494), (130.69633...","[(128.12, 588.13), (149.9, 588.77), (149.9, 61...",6/,"[[(128.12, 588.13), (149.9, 588.77)], [(149.9,...",6
3,"[(360.1, 799.08), (356.28, 799.16), (352.04, 7...",C,false,false,-1,None,unknown_1907_crop_2468_1674.jpg,"[(330.2155838013, 803.9237213135), (331.135627...",C,"[[(338.8154830933, 833.8014678955), (339.95793...","[17.047203063900042, -39.3532791138, -35.49494...","[[(338.8154830933, 833.8014678955), (339.95793...","[(330.22, 803.92), (331.14, 803.5), (332.28, 8...",C,"[[(330.2155838013, 803.9237213135), (331.13562...",C
4,"[(905.69, 83.16), (894.32, 83.48), (883.21, 83...",Hospital,false,false,-1,None,unknown_1907_crop_2468_1674.jpg,"[(828.5953674316, 84.6456375122), (832.4320983...",Hospital,"[[(828.54, 102.95), (905.22, 102.01)], [(905.6...","[77.15000000000009, -19.790000000000006, -76.3...","[[(828.54, 102.95), (905.22, 102.01)], [(905.6...","[(828.86, 84.52), (905.69, 83.16), (905.22, 10...",Hospital,"[[(828.86, 84.52), (905.69, 83.16)], [(905.22,...",Hospital
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,"[(840.49, 522.23), (858.24, 528.74), (853.21, ...",az,false,false,49,2,unknown_1907_crop_2468_1674.jpg,NaN,NaN,"[[(858.24, 528.74), (840.49, 522.23)], [(835.1...","[-23.08000000000004, 4.439999999999941, 12.720...","[[(835.16, 533.18), (853.21, 539.39)], [(858.2...","[(840.49, 522.23), (858.24, 528.74), (853.21, ...",AZ,"[[(840.49, 522.23), (858.24, 528.74)], [(853.2...",AZ
144,"[(794.33, 526.67), (845.81, 545.9), (843.45, 5...",ZEITUNE,false,false,49,3,unknown_1907_crop_2468_1674.jpg,"[(792.5183410645, 525.6031265259), (794.742759...",TEITUNE,"[[(845.81, 545.9), (794.33, 526.67)], [(789.3,...","[-56.50999999999999, -8.579999999999927, 49.12...","[[(789.3, 537.32), (843.45, 555.96)], [(845.81...","[(794.33, 526.67), (845.81, 545.9), (843.45, 5...",ZEITUNE,"[[(794.33, 526.67), (845.81, 545.9)], [(843.45...",ZEITUNE
145,"[(592.01, 414.55), (592.23, 412.73), (593.2, 4...",HÂRAT,false,false,50,1,unknown_1907_crop_2468_1674.jpg,"[(591.3165283203, 388.0397338867), (591.556427...",HARAT,"[[(590.93, 467.69), (592.01, 414.55), (592.32,...","[-13.469999999999914, -81.99000000000001, -16....","[[(577.46, 385.7), (576.29, 468.32)], [(590.93...","[(593.58, 387.92), (593.58, 465.95), (574.36, ...",HÂRAT/,"[[(593.58, 387.92), (593.58, 465.95)], [(574.3...",HÂRAT
146,"[(575.32, 519.34), (575.32, 519.34), (575.32, ...",al,false,false,50,2,unknown_1907_crop_2468_1674.jpg,NaN,NaN,"[[(575.32, 519.34), (574.85, 549.42)], [(589.4...","[14.120000000000005, 28.91999999999996, 13.370...","[[(575.32, 519.34), (574.85, 549.42)], [(589.4...","[(

In [15]:
df_merged[df_merged["group_id"]==1]

,vertices,text,illegible,truncated,group_id,sequence_order,crop_name,vertices_corrected,OCR_text,vertices_simplified,feature,vertices_simplified_order_corrected,vertices_new,text_new,vertices_final,text_final
23,"[(912.0, 429.08), (911.44, 439.28), (912.11, 4...",(MOUNT,false,false,1,1,unknown_1907_crop_2468_1674.jpg,"[(914.6887054443, 494.7551879883), (914.978088...",NOUET,"[[(914.1, 421.8), (912.0, 429.08), (913.7, 490...","[13.399999999999977, 73.59999999999997, 12.829...","[[(914.1, 421.8), (912.0, 429.08), (913.7, 490...","[(926.53, 423.34), (927.5, 495.4), (913.7, 490...",(MOUNT//,"[[(913.7, 490.0), (912.0, 429.08), (914.1, 421...",(MOUNT
24,"[(570.84, 312.72), (571.41, 304.97), (571.78, ...",hârat,false,false,1,1,unknown_1907_crop_2468_1674.jpg,NaN,NaN,"[[(570.84, 312.72), (573.75, 260.99)], [(559.8...","[-10.970000000000027, -52.870000000000005, -16...","[[(559.87, 259.85), (557.22, 311.62)], [(570.8...","[(573.75, 260.99), (570.84, 312.72), (557.22, ...",HÂRAT//,"[[(557.22, 311.62), (559.87, 259.85)], [(573.7...",HÂRAT
25,"[(914.84, 366.93), (914.6, 374.04), (913.93, 3...",SION,false,false,1,2,unknown_1907_crop_2468_1674.jpg,"[(911.770904541, 414.077835083), (911.95385742...",SON,"[[(926.7, 415.45), (928.56, 368.61)], [(914.84...","[-11.860000000000014, -48.51999999999998, -15....","[[(914.84, 366.93), (913.03, 414.89)], [(926.7...","[(928.56, 368.61), (926.7, 415.45), (913.03, 4...",SION//,"[[(913.03, 414.89), (914.84, 366.93)], [(928.5...",SION
26,"[(560.58, 252.28), (560.62, 255.11), (560.68, ...",al,false,false,1,2,unknown_1907_crop_2468_1674.jpg,NaN,NaN,"[[(573.55, 258.11), (573.12, 238.45)], [(560.6...","[-12.919999999999959, -20.580000000000013, -12...","[[(560.63, 237.53), (560.58, 252.28), (560.78,...","[(573.12, 238.45), (573.55, 258.11), (560.78, ...",AL//,"[[(560.78, 258.47), (560.58, 252.28), (560.63,...",AL
27,"[(920.47, 312.9), (919.55, 319.38), (918.74, 3...",STR.),false,false,1,3,unknown_1907_crop_2468_1674.jpg,"[(916.1359100342, 358.9216003418), (916.401794...",STR.,"[[(929.85, 360.41), (935.66, 316.17)], [(920.4...","[-9.379999999999995, -47.51000000000005, -20.1...","[[(920.47, 312.9), (915.5, 358.51)], [(929.85,...","[(935.66, 316.17), (929.85, 360.41), (915.5, 3...",STR.)//,"[[(915.5, 358.51), (920.47, 312.9)], [(935.66,...",STR.)
28,"[(560.87, 211.8), (560.9, 218.04), (560.53, 22...",kalâ,false,false,1,3,unknown_1907_crop_2468_1674.jpg,NaN,NaN,"[[(574.21, 236.37), (574.35, 193.65)], [(560.1...","[-14.06000000000006, -43.06999999999999, -13.7...","[[(560.15, 193.3), (560.87, 211.8), (560.58, 2...","[(574.35, 193.65), (574.21, 236.37), (560.58, ...",KALÂ//,"[[(560.58, 237.68), (560.87, 211.8), (560.15, ...",KALÂ
